In [ ]:
"""
README

code

	check_tensor.ipynb:
		demo code on how to load image tensor according to id

	|- data

		candidate_1m.json:
			dictionary of id and image_path in sorted id
			ignore contents for now, contents is the text pair of the image, it is not the search query
			example:
			{
			"id":0,
			"contents":"In New York City, Hundreds Become U.S. Citizens Just in Time to Vote",
			"image":"00000000_04639_01.jpg"
			}

		dev.json, test.json, train.json:
			26K/3.2K/3.2K random split of data
			example:
			    {
				"id":26217,
				"query":"GOP presidential candidate Ron Paul makes a point to Mitt Romney and Rick Santorum at the New Hampshire Republican debate",
				"candidates":[
					{
						"candidate_id":181181,
						"headline":"Mitt Romney: wins He is so far ahead in the polls it would be one of the biggest upsets in US political history if he were to lose",
						"image":"00181181_40000_03.jpg",
						"meta_source":"ground_truth",
						"score":3,
						"source":"theguardian.com",
						"vn_id":608630
					},
					... SKIP MIDDLE
					{
						"article":"https://www.politico.com/story/2012/01/mitt-unscathed-as-pack-squabbles-071198",
						"candidate_id":181188,
						"headline":"Mitt unscathed as pack squabbles",
						"image":"00181188_26217_00.jpg",
						"image_url":"https://www.politico.com/dims4/default/38d41ea/2147483647/legacy_thumbnail/1200x800%3E/quality/90/?url=http%3A%2F%2Fs3-origin-images.politico.com%2F2012%2F01%2F120107_romney_debate_ap_3281.jpg",
						"meta_source":"google_search",
						"score":2,
						"source":"politico.com"
					}
				]
				}

"""

In [1]:
import json
import torch
import clip
from PIL import Image
import collections
import operator
from collections import OrderedDict
import collections
from tqdm import tqdm
import os
import numpy as np

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
save_dir = "data/tensor/"
images_load = np.load(save_dir+"images.npy")
image_np = np.asarray(images_load)
image_tensors_load = torch.from_numpy(image_np)
image_tensors_load = torch.squeeze(image_tensors_load)
image_tensors_load = image_tensors_load.type(torch.float16)
print(type(image_tensors_load))
print(image_tensors_load.size())

<class 'torch.Tensor'>
torch.Size([1040919, 512])


In [7]:
# file_name {"dev.json","test.json","train.json"}
file_name = "dev.json"
doc_lst = json.load(open("data/"+file_name))
dim = len(doc_lst)

In [8]:
device = "cuda:2" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print("loading success")

loading success


In [9]:
INDEX = 200

candidate = doc_lst[INDEX]["candidates"][0]
id = candidate["candidate_id"]
image_path = candidate["image"]
headline = candidate["headline"]
print(image_path)

00064768_11448_01.jpg


******************  

NOTICE:   
below you will not be able to run unless download actual images dataset

******************  

In [19]:
path = "/data2/weixifeng/MMR/dataset/images/"
# try:
image = preprocess(Image.open(path+image_path)).unsqueeze(0).to(device)
with torch.no_grad():
	image_feature = model.encode_image(image)
	image_feature /= image_feature.norm(dim=-1, keepdim=True).to(device)
# except:
# 	print("skipped")
# 	image_feature = torch.empty(1,512).to(device)
print(image_feature.size())
print(image_feature)

torch.Size([1, 512])
tensor([[ 1.0651e-02, -2.8763e-02, -7.5340e-03, -3.6287e-04,  5.1086e-02,
          1.6663e-02,  5.9296e-02, -7.5722e-03,  3.1403e-02, -1.0696e-02,
          5.6038e-03, -2.3758e-02,  3.9551e-02, -2.3285e-02,  9.6741e-02,
         -3.4698e-02, -2.5253e-02, -1.6281e-02, -1.4252e-02, -1.1234e-03,
         -2.5620e-02,  6.0959e-03,  8.6212e-03, -4.3365e-02, -2.8107e-02,
          4.0527e-02,  4.7089e-02,  1.2108e-02,  2.0355e-02, -3.1494e-02,
         -1.2001e-02,  2.3575e-02, -1.3069e-02, -1.0223e-02, -3.8948e-03,
         -6.5863e-05,  3.3600e-02,  2.2621e-03, -1.8997e-02,  1.1932e-01,
         -5.4535e-02, -1.8433e-02,  9.9373e-04, -4.4708e-02, -1.3275e-02,
          8.4717e-02,  6.4964e-03,  1.7456e-02, -3.0075e-02, -3.4363e-02,
          3.3600e-02, -1.5427e-02, -3.3989e-03,  4.3854e-02, -9.5520e-03,
          3.4851e-02, -2.3132e-02,  3.1311e-02,  2.1118e-02, -6.7139e-03,
          3.1555e-02, -7.1487e-03,  2.4628e-02,  1.5778e-02, -3.0838e-02,
          2.5085e

In [20]:
print(image_tensors_load[id])

tensor([ 1.0689e-02, -2.8748e-02, -7.4615e-03, -4.0412e-04,  5.1056e-02,
         1.6678e-02,  5.9296e-02, -7.5798e-03,  3.1311e-02, -1.0651e-02,
         5.6725e-03, -2.3544e-02,  3.9673e-02, -2.3376e-02,  9.6802e-02,
        -3.4637e-02, -2.5208e-02, -1.6190e-02, -1.4374e-02, -1.1244e-03,
        -2.5452e-02,  5.9891e-03,  8.7051e-03, -4.3243e-02, -2.8198e-02,
         4.0436e-02,  4.7211e-02,  1.2306e-02,  2.0279e-02, -3.1525e-02,
        -1.1993e-02,  2.3529e-02, -1.3092e-02, -1.0246e-02, -3.9024e-03,
        -1.2875e-05,  3.3600e-02,  2.1896e-03, -1.9104e-02,  1.1920e-01,
        -5.4535e-02, -1.8372e-02,  1.0958e-03, -4.4861e-02, -1.3367e-02,
         8.4900e-02,  6.3629e-03,  1.7502e-02, -3.0273e-02, -3.4363e-02,
         3.3569e-02, -1.5480e-02, -3.2864e-03,  4.3976e-02, -9.4604e-03,
         3.4821e-02, -2.3315e-02,  3.1311e-02,  2.1103e-02, -6.8169e-03,
         3.1342e-02, -7.1373e-03,  2.4597e-02,  1.5869e-02, -3.0853e-02,
         2.5040e-02,  4.3213e-02,  7.7026e-02, -6.3

In [ ]:
# image_tensors = []

# skip = 0

# for data in tqdm(doc_lst):
# 	img_name = data["image"]
# 	if img_name[0] == '.':
# 		image_path = vn_dir+img_name
# 	else:
# 		image_path = tara_dir+img_name
# 	try:
# 		image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)		
# 		with torch.no_grad():
# 			image_feature = model.encode_image(image)
# 			image_feature /= image_feature.norm(dim=-1, keepdim=True).to(device)
# 	except:
# 		skip += 1
# 		print("skipped")
# 		image_feature = torch.empty(1,512).to(device)
# 	# print(type(image_feature),image_feature.shape)
# 	image_tensors.append(image_feature)

# print("skip count",skip)

# image_tensors = torch.stack(image_tensors)
# image_tensors = torch.squeeze(image_tensors)

# print("Image tensor",image_tensors.shape)
# # save_dir = "/mnt/edward/data2/emeraldliu/datas/model/clip/tensors/"
# image_tensors = image_tensors.reshape(dim,1,512)
# var_path = os.path.join(save_dir+"/images.npy")
# np_var = image_tensors.data.cpu().numpy()
# np.save(var_path,np_var)
# images_load = np.load(save_dir+"/images.npy")
# image_np = np.asarray(images_load)
# image_tensors_load = torch.from_numpy(image_np)
# image_tensors_load = torch.squeeze(image_tensors_load)
# image_tensors_load = image_tensors_load.to(device)
# print("Image tensor",image_tensors_load.shape)